In [1]:
#First we import the libraries we will need
import urllib
import urllib2
import time
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#First of all we need to find all the name of the sites that belong to fortune 500. This can happen if we seperate
#The information needed from the below link
url = "http://www.zyxware.com/articles/4344/list-of-fortune-500-companies-and-their-websites"
list_company_number =[]
list_company_name = []
list_company_website = []
list500_sites = []
list500_names = []
list500_num = []
list500_url = []

In [3]:
#In order to extract the needed informations we will create 3 lists. The first one will contain the rank of each site, the
#second one will contain the name of the company and the 3rd one will contain the actual link of the company's site.
#For achieving this purpose we will create a funstion that will in its turn create those three list.
#In order to know if the function worked we will ask it to return the first element of each list along with a sentence.
def websites (url): 
    from time import time # I used it to see how much time it does to run the function
    start = time ()
    browser = urllib2.build_opener() 
    browser.addheaders = [('User-agent', 'Mozilla/5.0')]
    response = browser.open(url)# this might throw an exception if something goes wrong.
    myHTML = response.read()
    soup = BeautifulSoup(myHTML,"lxml")    
    o = 0
    td_list =[]
    for row2 in soup.html.body.findAll('td'):
        td_list.insert(o, row2)
        o = o + 1
    a = 0
    b = 1
    c = 2
    list_numbering = 0
    for i in range (0,500):        
        num = str(td_list[a])
        company = str(td_list[b])
        site = str(td_list[c])
        c_num = re.findall('>(.+?)</td>',num)  
        c_num = str(c_num[0])
        c_name = re.findall('>(.+?)</td>',company)
        c_name = str(c_name[0])
        c_site = re.findall('">(.+?)</a>',site)
        c_site = str(c_site[0])        
        list_company_number.insert(list_numbering,c_num)
        list_company_name.insert(list_numbering,c_name)
        list_company_website.insert(list_numbering,c_site)
        a = a + 3
        b = b + 3
        c = c + 3
        list_numbering =  list_numbering + 1 
    end = time ()
    duration = round (end - start, 1)
    minutes = round (duration /60, 1)
    print 'The lists are ready in ', duration, ' seconds'

In [4]:
# After creating the function we should now test that it actually works correctly
websites (url)

The lists are ready in  1.4  seconds


In [5]:
#Try to validate each page url #pip install validators
import validators
nv = 0
for num in range(len(list_company_website)):
    line = 'http://' + str(list_company_website[num])
    x = validators.url(line)    
    if x != True:
        nv = nv +1
print "The validation is complete! There were" , nv, "not valid pages"

The validation is complete! There were 0 not valid pages


In [6]:
## The next step is to download and save in a new list the 500 sites.
## We will create a new function that will do this by reading the list "list_company_websites" that we created 
## with the previous function

def list_company_HTML (list_company_website,list_company_name,start,end):
    import time
    browser2 = urllib2.build_opener()
    browser2.addheaders = [('User-agent', 'Mozilla/5.0')]
    for i in range (start,end):
        k = str(i + 1)
        lc = str(list_company_website[i])        
        lc = lc.replace("'","")   
        lc = lc.replace("[","")
        lc = lc.replace("]","")
        lcn = str(list_company_name[i])        
        lcn = lcn.replace("'","")   
        lcn = lcn.replace("[","")
        lcn = lcn.replace("]","")
        url2= 'http://' + lc
        #an exception might be thrown, so the code should be in a try-except block
        try:            
            response2=browser2.open(url2)
        except Exception: # this describes what to do if an exception is thrown
             continue     
        #read the response in html format. This is essentially a long piece of text
        myHTML2=response2.read()
        list500_sites.insert(i,myHTML2)
        list500_names.insert(i,lcn)
        list500_url.insert(i,lc)
        list500_num.insert(i,k)
        #wait for 2 seconds
        time.sleep(2)
        #print ("The site " + k + " has been downloaded!")
    print "We downloaded: ",len(list500_sites)," sites!"
    #print (len(list500_names),list500_names)

In [7]:
start = 0
stop = 25
for i in range (0,1): #i am not downloading all the pages for the time being for reasons of running faster the process during the tests
    print (start)
    print (stop)    
    list_company_HTML (list_company_website,list_company_name,start,stop)
    start = stop
    stop = stop + 25

0
25
We downloaded:  24  sites!


In [8]:
#As we can see there is one site that hasn't been downloaded in order to keep track of the sites that we could not download
#we will create a new list that we will keep them all together there
not_d = []
def not_downloadables (list500_names,list_company_name):
    met = 0
    for i in range(0,25):#normally it would be till 500 but now we test for the first 25 we have downloaded
        ct = list_company_name[i]
        if ct not in list500_names:
            not_d.insert(met,ct)
    print not_d        

In [9]:
#Now we will run the function to see which sites haven;t been downloaded
not_downloadables (list500_names,list_company_name)

['Fannie Mae']


In [10]:
#Now we will perfom a reliability test for the text that is included in the html code of the company's page
from pattern import metrics
readability = []
rdb = []

In [11]:
def readable (list500_names,list500_sites):
    for i in range (len(list500_names)):
        myHTML = list500_sites[i]
        a = metrics.flesch_reading_ease(myHTML) * 100
        a = round (a, 1)
        if a > 90:    
            readability.insert(i,"Very easy")
            rdb.insert(i,6)
        elif a > 80:
            readability.insert(i,"Easy")
            rdb.insert(i,5)
        elif a > 70:
            readability.insert(i,"Fairly easy")
            rdb.insert(i,4)
        elif a > 60:
            readability.insert(i,"Standard")
            rdb.insert(i,3)
        elif a > 50:
            readability.insert(i,"Fairly difficult")
            rdb.insert(i,2)
        elif a > 30:
            readability.insert(i,"Difficult")
            rdb.insert(i,1)
        else:
            readability.insert(i,"Very Confusing")
            rdb.insert(i,0)
    print "The function is completed!"

In [12]:
readable (list500_names,list500_sites)

The function is completed!


In [13]:
d1 = {'company' : pd.Series(list500_names, index=[list500_num]),'url' : pd.Series(list500_url, index=[list500_num]),
      'Readability' : pd.Series(readability, index=[list500_num]),'Readability_index' : pd.Series(rdb, index=[list500_num])}
fre = pd.DataFrame(d1)    
fre #.head(3) #we see the first 3 in the data frame

,Readability,Readability_index,company,url
1,Very Confusing,0,Walmart,www.walmart.com
2,Difficult,1,Exxon Mobil,www.exxonmobil.com
3,Very Confusing,0,Apple,www.apple.com
4,Very Confusing,0,Berkshire Hathaway,www.berkshirehathaway.com
5,Very Confusing,0,McKesson,www.mckesson.com
6,Very Confusing,0,UnitedHealth Group,www.unitedhealthgroup.com
7,Very Confusing,0,CVS Health,www.cvshealth.com
8,Very Confusing,0,General Motors,www.gm.com
9,Very Confusing,0,Ford Motor,www.ford.com
10,Very Confusing,0,AT&amp;T,www.att.com


In [18]:
fre.to_csv('my_csv.csv', sep='\t')

In [14]:
#Retreiving the social media from each site
#First create empty lists for the ones that we will need to calculate
sm_f = []
sm_t = []
sm_i = []
sm_p = []
sm_y = []
sm_l = []   
sm_nm = [] 
nm = []
sm_url = []

In [15]:
#Then create a function that will feel in those lists so as to make the data frame later on

def socialmedia (list500_sites,list500_names,list500_url):
    from time import time # I used it to see how much time it does to run the function
    start = time ()
    for num in range(len(list500_names)):
        if list500_sites[num] !=  "":
            hand = str(list500_sites[num])
            sm = ['facebook.com','twitter.com','instagram.com','pinterest.com','youtube.com','linkedin.com'] 
            number = 0 
            for index in range(len(sm)):
                x = sm[index]
                photo = re.findall(x,hand)                                
                if (len(photo) > 0):
                    if x == 'facebook.com':
                        answerf = 'yes'
                    if x == 'twitter.com':
                        answert = 'yes'
                    if x == 'instagram.com':
                        answeri = 'yes'
                    if x == 'pinterest.com':
                        answerp = 'yes'
                    if x == 'youtube.com':
                        answery = 'yes'
                    if x =='linkedin.com':
                        answerl = 'yes'                   
                else:
                     if x == 'facebook.com':
                        answerf = 'no'
                     if x == 'twitter.com':
                        answert = 'no'
                     if x == 'instagram.com':
                        answeri = 'no'
                     if x == 'pinterest.com':
                        answerp = 'no'
                     if x == 'youtube.com':
                        answery = 'no'
                     if x =='linkedin.com':
                        answerl = 'no'                 
            sm_nm.insert(num,list500_names[num]) 
            nm.insert(num,num)
            sm_url.insert(num,list500_url[num])
            sm_f.insert(num,answerf)
            sm_t.insert(num,answert)
            sm_i.insert(num,answeri)
            sm_p.insert(num,answerp)
            sm_y.insert(num,answery)
            sm_l.insert(num,answerl)
    end = time ()
    duration = round (end - start, 3)
    minutes = round (duration /60, 1)
    print 'The lists are completed in ', minutes, ' minutes' 
    print 'The lists are ready in ', duration, ' seconds'

In [16]:
#Now we will run the function for the 25 first sites for starters
socialmedia (list500_sites,list500_names,list500_url)

The lists are completed in  0.0  minutes
The lists are ready in  0.021  seconds


In [17]:
#Finally we create the data frame with the elements we found            
d2 = {'company' : pd.Series(sm_nm, index=[nm]),'url' : pd.Series(sm_url, index=[nm]),
     'facebook' : pd.Series(sm_f, index=[nm]),'twitter' : pd.Series(sm_t, index=[nm]),
     'instagram' : pd.Series(sm_i, index=[nm]),'pinterest' : pd.Series(sm_p, index=[nm]),
     'youtube' : pd.Series(sm_y, index=[nm]),'linkedin' : pd.Series(sm_l, index=[nm]),}
social_media = pd.DataFrame(d2)    
social_media.head(3) #we see the first 3 in the data frame

,company,facebook,instagram,linkedin,pinterest,twitter,url,youtube
0,Walmart,yes,yes,no,yes,yes,www.walmart.com,yes
1,Exxon Mobil,yes,no,no,no,no,www.exxonmobil.com,no
2,Apple,no,no,yes,no,no,www.apple.com,yes


In [18]:
#Create the lists we will need for the data frame
l_nm = []
l_ex = []
l_in = []
l_t = []
nm = []
l_url = []

In [19]:
#create the function that will calculate the different type of links
def links (list500_sites,list500_names,list500_url):
    from time import time # I used it to see how much time it does to run the function
    start = time ()
    for num in range(len(list500_names)):
        line = str(list500_sites[num])
        href = re.findall('href',line)
        external = re.findall('href="https:',line)
        ex = (len(external))
        alllinks = (len(href))
        internal =  (len(href) - len(external))
        l_nm.insert(num,list500_names[num])            
        l_ex.insert(num,ex)
        l_t.insert(num,alllinks)
        l_in.insert(num,internal)
        nm.insert(num,num)
        l_url.insert(num,list500_url[num])
    end = time ()
    duration = round (end - start, 3)
    minutes = round (duration /60, 1)
    print 'The lists are ready in ', minutes, ' minutes'
    print 'The lists are ready in ', duration, ' seconds'

In [20]:
#Run the function in order to find the external, internal and total links of each site
#For now we are running for the first 25 sites only
links (list500_sites,list500_names,list500_url)

The lists are ready in  0.0  minutes
The lists are ready in  0.007  seconds


In [21]:
#Create a dataframe so as to be able to see the results of the function we run
d3 = {'company' : pd.Series(l_nm, index=[nm]),'url' : pd.Series(l_url, index=[nm]),
      'external' : pd.Series(l_ex, index=[nm]),'internal' : pd.Series(l_in, index=[nm]),
     'total links' : pd.Series(l_t, index=[nm])}
sites_links = pd.DataFrame(d3)    
sites_links.head(3) #we see the first 3 in the data frame

,company,external,internal,total links,url
0,Walmart,47,122,169,www.walmart.com
1,Exxon Mobil,23,784,807,www.exxonmobil.com
2,Apple,6,276,282,www.apple.com


In [22]:
#The initial lists we will need in order to calculate the loading time
lt_nm = [] 
lt_time = []
nm = []
lt_url = []

In [23]:
#the function that will calculate the loading time
def loadtime (list_company_website,list500_names,list500_url):
    from time import time
    browser2 = urllib2.build_opener()
    browser2.addheaders = [('User-agent', 'Mozilla/5.0')]
    for num in range(len(list500_names)):
        lc = str(list_company_website[num])        
        lc = lc.replace("'","")   
        lc = lc.replace("[","")
        lc = lc.replace("]","")
        url2 = 'http://' + lc        
        try:
            response2 = browser2.open(url2)
        except Exception: 
             continue#     
        start_time = time()
        myHTML2 = response2.read()
        end_time = time()
        response2.close()
        l_t = round(end_time-start_time, 3) #in order to be more readable we rounded the time
        loadt = str(l_t)
        lt_nm.insert(num,list500_names[num])            
        lt_time.insert(num,loadt)
        nm.insert(num,num)
        lt_url.insert(num,list500_url[num])
    print "The function is completed!"

In [24]:
#running the function for the first 25 sites
loadtime (list_company_website,list500_names,list500_url)

The function is completed!


In [25]:
#creating the data frame with the loading times
d4 = {'company' : pd.Series(lt_nm, index=[nm]),'url' : pd.Series(lt_url, index=[nm]),
      'loading time' : pd.Series(lt_time, index=[nm])}
loading_time = pd.DataFrame(d4)    
loading_time.head(3) #we see the first 3 in the data frame

,company,loading time,url
0,Walmart,0.443,www.walmart.com
1,Exxon Mobil,3.629,www.exxonmobil.com
2,Apple,0.075,www.apple.com


In [26]:
#Find out how many and what type of images each site has
#first we create the initially empty lists
p_p = []
p_d = []
p_jpg = []
p_jpeg = []
p_gif = []
p_tif = []
p_tiff = []
p_bmp = []
p_jpe = []
p_nm = []
p_tt =[]
nm = []
p_url = []

In [27]:
#Then we create the function that will explore the html pages and search for the images
def images (list500_sites,list500_names,list500_url):
    from time import time # I used it to see how much time it does to run the function
    start = time ()
    for num in range(len(list500_names)):
        line = str(list500_sites[num])
        image = ['.png','.dib','.jpg','.jpeg','.bmp','.jpe','.gif','.tif','.tiff'] 
        totalnumber = 0 
        for index in range(len(image)):
            x = image[index]
            photo = re.findall(x,line)
            if x == '.png':
                p = str (len(photo))
            if x == '.dib':
                d = str (len(photo))
            if x == '.jpg':
                jpg = str (len(photo))
            if x == '.jpeg':
                jpeg = str (len(photo))
            if x == '.gif':
                gif = str (len(photo))
            if x == '.tif':
                tif = str (len(photo))
            if x == '.tiff':
                tiff = str (len(photo))
            if x == '.bmp':
                bmp = str (len(photo))
            if x == '.jpe':
                jpe = str (len(photo))
            totalnumber = len(photo) + totalnumber
        total = str (totalnumber)
        p_nm.insert(num,list500_names[num])            
        p_p.insert(num,p)  
        p_d.insert(num,d)  
        p_jpg.insert(num,jpg)  
        p_jpeg.insert(num,jpeg)  
        p_gif.insert(num,gif)  
        p_tif.insert(num,tif)  
        p_tiff.insert(num,tiff)  
        p_bmp.insert(num,bmp)  
        p_jpe.insert(num,jpe)  
        p_tt.insert(num,total)
        nm.insert(num,num)
        p_url.insert(num,list500_url[num])
    end = time ()
    duration = round (end - start, 3)
    minutes = round (duration /60, 1)
    print 'The lists are ready in ', minutes, ' minutes'
    print 'The lists are ready in ', duration, ' seconds'

In [28]:
#Then we run the function for the first 20 sites for now
images (list500_sites,list500_names,list500_url)

The lists are ready in  0.0  minutes
The lists are ready in  0.238  seconds


In [29]:
#Finally we create a dataframe in order to see the results of the function
d5 = {'company' : pd.Series(p_nm, index=[nm]),'url' : pd.Series(p_url, index=[nm]),
      '.png' : pd.Series(p_p, index=[nm]),'.dib' : pd.Series(p_d, index=[nm]),
'.jpg' : pd.Series(p_jpg, index=[nm]),'.jpeg' : pd.Series(p_jpeg, index=[nm]),
'.bmp' : pd.Series(p_bmp, index=[nm]),'.jpe' : pd.Series(p_jpe, index=[nm]),
'.gif' : pd.Series(p_gif, index=[nm]),'.tif' : pd.Series(p_tif, index=[nm]),
'.tiff' : pd.Series(p_tiff, index=[nm]), 'total images' : pd.Series(p_tt, index=[nm])}
images_types = pd.DataFrame(d5)    
images_types.head(3) #we see the first 3 in the data frame

,.bmp,.dib,.gif,.jpe,.jpeg,.jpg,.png,.tif,.tiff,company,total images,url
0,0,0,26,103,103,85,43,12,0,Walmart,372,www.walmart.com
1,0,0,1,0,0,16,2,4,0,Exxon Mobil,23,www.exxonmobil.com
2,0,0,1,0,0,0,2,0,0,Apple,3,www.apple.com


In [30]:
#Now we will find the different dimensions that each site uses
#initially we create the empty lists we will need
nm = []
s_comp = []
s_dimensions = []
s_times = []
s_tt_dif_dim = []
ht = [] #list of different heights in each case
wt = [] #list of different widths in each case
h_w = [] # combinations of height and width
dif_size = []  
un_size = [] 
s_url = []

In [31]:
#With the below function we will gather in a variable all the different dimensions 
#and in another one all the times that each dimension occures for each html code
def find_dif_sizes (list500_sites,list500_names,list500_url):
    from time import time # I used it to see how much time it does to run the function
    start = time ()
    for num in range(len(list500_names)):
        nm.insert(num,num)                  
        s_comp.insert(num,list500_names[num])
        s_url.insert(num,list500_url[num])
        line = str(list500_sites[num]) #read each line of the list500_sites that contains each html code
        soup = BeautifulSoup(line, "lxml")
        # we create 2 local variables so as to gather the different dimensions and occurencies  of each page seperately
        s_dimensions_local = []
        s_times_local = []
        hw = 0 # we use it for the lists of height and width
        for tag in soup.find_all('img'): # find all the img in the first site html
            #Since in some cases either the height or the width is missing we would like to keep only the ones that have both dimensions
             h = tag.attrs.get('height', None)
             w = tag.attrs.get('width', None)
             #we use if to check which ones have both        
             if h != None:
                 if w != None:
                     ht.insert(hw,h)
                     wt.insert(hw,w)
                     hw = hw + 1                        
        hw2 = 0
        for l in range(len(ht)):
             h_w_c = ht[l] + 'x' + wt[l]    #we create a str with the form (300x300) so as to be more easily to read later on 
             h_w.insert(hw2,h_w_c)  #we put it in a new list
             hw2 = hw2 + 1    
        if h_w == []:#we check if there are not any dimensions available
             nm.insert(num,num)                  
             s_comp.insert(num,list500_names[num])
             s_dimensions.insert(num,0)
             s_times.insert(num,0)    
        if h_w != []:#now we continue with the cases where the dimensions are indeed available             
             from collections import Counter
             hw_unique = Counter(h_w)
             hw_unique2 = str(hw_unique) #the unique different dimensions for the specific site
            #Due to the fact that we are talking about a list we have to split the parts we need 
             split1 = hw_unique2.split('{')
             a = split1[1]
             split2 = a.split('}')
             b = split2[0]
             split3 = b.split(',')
             finalsplit = []
             fs = []
             z = 0
             m = 1
             j = 0
             z1 = 0
             m1 = 1
            #each of the items in split3 has a form '300x300 : 15' and in order to create the dataframe we have 
            #to split this form and keep the informations in different list
             for numb in split3:                
                 oldstring = numb
                 newstring = oldstring.replace("'", "")
                 new = newstring.replace("'","")
                 string = new.replace(" ","")
                 finalstring = string.split(':')
                    #the finalstring is a list that contains the dimensions and the occurencies
                    #in order toseperate in different lists we create an additional loop
                 for xx in range(len(finalstring)):
                     ax = finalstring[xx]
                     if 'x' in ax:
                         s_dimensions_local.insert(z1,finalstring[xx])
                         z1 = z1 + 1
                     else:
                         s_times_local.insert(m1,finalstring[xx])
                         m1 = m1 + 1  
            #Now we can add to the lists the parts we created so as to have them all gathered together             
             s_dimensions.insert(num,s_dimensions_local)
             s_times.insert(num,s_times_local)                
    end = time ()
    duration = round (end - start, 3)
    minutes = round (duration /60, 1)
    print 'The lists are ready in ', minutes, ' minutes'
    print 'The lists are ready in ', duration, ' seconds'

In [32]:
#Run the function for the first 20 sites
find_dif_sizes (list500_sites,list500_names,list500_url)

The lists are ready in  0.0  minutes
The lists are ready in  0.783  seconds


In [33]:
#Find the unique different image dimensions and put them on a list
def unique_dif_sizes (s_dimensions,list500_names):
    ds = 0
    for num in range(len(list500_names)):
        for s in s_dimensions[num]:
            dif_size.insert(ds,s)
            ds = ds + 1
    dsu = 0
    for i in dif_size:
        if i not in un_size:
            un_size.insert(dsu,i)
            dsu = dsu + 1
    print(un_size)        

In [34]:
#Run the finction unique_dif_sizes
unique_dif_sizes (s_dimensions,list500_names)

['144x144', '15x75', '44x556', '1x1', '800x1200', '24pxx133px', '21pxx173px', '70x125', '50x45', '400x300', '292pxx292px', '200pxx200px', '300pxx1500px', '1279pxx984px', '29x29']


In [35]:
#The lists we will need for the next function
t_f_s = []
ttf = []

In [36]:
#Function in order to check whether or not each company has these dimensions
def dimensions_per_company (un_size,list500_names):
    from time import time # I used it to see how much time it does to run the function
    start = time ()
    t_f_s.insert(0,un_size)
    ttf.insert(0,t_f_s)
    for num in range(len(list500_names)):
        s1a = s_dimensions[num] #dimensions of site num
        where = [] #empty list
        wh = 0
        haveornot = []
        for er in range (len(un_size)):
            for sizea in s1a:
                if sizea == un_size[er]:
                    where.insert(wh,str(er))
                    break
            if str(er) in where:
               haveornot.insert(er,True)                    
            else:
               haveornot.insert(er,False)
                    
        t_f_s.insert(num + 1,haveornot)
        ttf.insert(num + 1,t_f_s)  
    end = time ()
    duration = round (end - start, 3)
    minutes = round (duration /60, 1)
    print 'The lists are ready in ', minutes, ' minutes'
    print 'The lists are ready in ', duration, ' seconds'

In [37]:
#Run the function dimensions_per_company
dimensions_per_company (un_size,list500_names)

The lists are ready in  0.0  minutes
The lists are ready in  0.001  seconds


In [38]:
print(s_comp)

['Walmart', 'Exxon Mobil', 'Apple', 'Berkshire Hathaway', 'McKesson', 'UnitedHealth Group', 'CVS Health', 'General Motors', 'Ford Motor', 'AT&amp;T', 'General Electric', 'AmerisourceBergen', 'Verizon', 'Chevron', 'Costco', 'Kroger', 'Amazon.com', 'Walgreens Boots Alliance', 'HP', 'Cardinal Health', 'Express Scripts Holding', 'J.P. Morgan Chase', 'Boeing', 'Microsoft']


In [39]:
#Create an initial dataframe where we will add the sizes later on
d6 = {'company' : pd.Series(s_comp, index=[nm]),'url' : pd.Series(s_url, index=[nm])}
sizess = pd.DataFrame(d6)    
sizess.head(3)

,company,url
0,Walmart,www.walmart.com
1,Exxon Mobil,www.exxonmobil.com
2,Apple,www.apple.com


In [40]:
#Now we want to break the variable t_f_s in order to add the columns to the dataframe                  
#Finally we create the data frame with the elements we found 
def final_dimensions_dataframe (un_size,t_f_s,list500_names):
    from time import time # I used it to see how much time it does to run the function
    start = time ()
    for q in range(len(un_size)):
        names = un_size[q]
        var = []
        for num in range(len(list500_names)):
            a = t_f_s[num+1]
            var.insert(num,a[q])
        sizess[names] = pd.Series(var, index=sizess.index) 
    end = time ()
    duration = round (end - start, 3)
    minutes = round (duration /60, 1)
    print 'The lists are ready in ', minutes, ' minutes'
    print 'The lists are ready in ', duration, ' seconds'

In [41]:
#Run the function final_dimensions_dataframe
final_dimensions_dataframe (un_size,t_f_s,list500_names)

The lists are ready in  0.0  minutes
The lists are ready in  0.004  seconds


In [42]:
sizess.head(3)

,company,url,144x144,15x75,44x556,1x1,800x1200,24pxx133px,21pxx173px,70x125,50x45,400x300,292pxx292px,200pxx200px,300pxx1500px,1279pxx984px,29x29
0,Walmart,www.walmart.com,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,Exxon Mobil,www.exxonmobil.com,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,Apple,www.apple.com,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [43]:
#Now we would like to find the words in the text and the unique words of each html page
#First of all we need to have a dictionary with which we would check if the word we found truly exists
#The dictionary is available in the internet from a github acount from where we are going to read it
url_dictionary = "https://raw.githubusercontent.com/dwyl/english-words/master/words.txt"
browser = urllib2.build_opener()
browser.addheaders = [('User-agent', 'Mozilla/5.0')]
response = browser.open(url_dictionary)
html_dictionary = response.read()
html_dictionary
dicti = str(html_dictionary)
#dicti

In [44]:
#dict_new = dicti.split("\\n")
dict_new = dicti.split("\n")
dict_new[49] #the first 49 parts are not words so we have to remove them from the list

'a1'

In [45]:
dict_final = []
df = 0
for i in range (50,len(dict_new)):
    forfinal = dict_new[i]
    forfinal = forfinal.replace("'","")
    dict_final.insert(df,forfinal)
    df = df + 1
dict_final[0] #This is the original dictionary with which we will check each word

'aa'

In [48]:
#And now we will find each html file which words has inside
empty = []
wordsin = []
ocin = []
def html_which_word (list500_sites):
    from time import time # I used it to see how much time it does to run the function
    start_t = time()
    for num in range(len(list500_sites)):
        line = str(list500_sites[num]) 
        wordcount={}
        corwordcount={}
        simeiastiksis = ["/",".",",","=",">","<","?","|",":","_","]","[","$","&","%","(",")","{","}",'"',";","\\","-","!","+","#","=","@","^","*","'"]
        for ss in range(len(simeiastiksis)):
            simeio = simeiastiksis[ss]     
            line = line.replace(simeio, " ")
        for word1 in line.split():
            word1 = word1.lower()
            if word1 in dict_final:
                if word1 not in wordcount:
                    wordcount[word1] = 1
                else:
                    wordcount[word1] += 1     
        wordsin_local = []
        wl = 0
        ocin_local = []
        for k,v in wordcount.items():
               #print (k,v)
               wordsin_local.insert(wl,str(k))
               ocin_local.insert(wl,str(v))
               wl = wl + 1
        wordsin.insert(num,wordsin_local) # final list with all the words in each site
        ocin.insert(num,ocin_local)  #final list with all the occurencies of the words in each site
    end_t = time()
    total_t = round(end_t - start_t,3)
    total_ = round(total_t / 60,1)
    print('finished ',str(x) ,' sites in: ', str(total_),' minutes')

In [49]:
html_which_word (list500_sites) #test the code for 2 sites

('finished ', 'True', ' sites in: ', '12.2', ' minutes')


In [50]:
#Create the dataframe for the words and unique words
d7 = {'company' : pd.Series(list500_names, index=[nm]),'url' : pd.Series(list500_url, index=[nm])}
wordss = pd.DataFrame(d7)    
wordss.head(3) 

,company,url
0,Walmart,www.walmart.com
1,Exxon Mobil,www.exxonmobil.com
2,Apple,www.apple.com


In [52]:
#Create the two lists we will need in order to make the dataframe
l1 = []
l2 = []
for num in range(len(list500_sites)):   #for testing resons
    line = str(list500_sites[num]) 
    total_words = len(wordsin[num])
    occurencies = ocin[num] 
    l1.insert(num,total_words)
    count = 0 
    for a in occurencies :
        if a == '1':
            count = count + 1
    l2.insert(num,count)
wordss['total_words'] = pd.Series(l1, index=sizess.index)
wordss['unique_words'] = pd.Series(l2, index=sizess.index)          
wordss.head(3)                 

,company,url,total_words,unique_words
0,Walmart,www.walmart.com,1501,415
1,Exxon Mobil,www.exxonmobil.com,801,151
2,Apple,www.apple.com,399,136


In [53]:
#In order to validate the html code we will use the w3 validator
#We will validate each url and then we will open the url of the validation page
#so as to extract the errors, the info warnings and the non-document-error io informations 
#First we create the empty lists we would use later on
num_errors = []
num_info_warnings = []
num_non_doc = [] 
nm = []
num_open_page = []
empty = ""

In [54]:
#Then we create the function that will pull the informations we want
def html_validation (list500_url,list500_names):
    from time import time # I used it to see how much time it does to run the function
    start = time ()
    for num in range(len(list500_names)):
        url_check = "https://validator.w3.org/nu/?doc=https://" + list500_url[num]
        browser = urllib2.build_opener()
        browser.addheaders = [('User-agent', 'Mozilla/5.0')]
        response = browser.open(url_check)
        html_check = response.read()
        html_check
        check = str(html_check)
        er = 0
        err = 0
        errr = 0
        e = False
        if check != empty:
            e = True
            soup = BeautifulSoup(check,"lxml")
            o = 0
            keyf = []
            for row in soup.html.body.findAll('div'):
                keyf.insert(o,row)
                o = o + 1
            #print(len(keyf),list500_url[num], "site number: ", str(num))        
            if len(keyf) != 0:       
                    keyfin = str(keyf[2]) #the elements we need is in the 2nd div of the code
                    dol= re.findall('class="error"',keyfin)            
                    er = er + len(dol)
                    doll= re.findall('class="info warning"',keyfin)            
                    err = err + len(doll)
                    dolll= re.findall('class="non-document-error io"',keyfin)            
                    errr = errr + len(dolll)
        num_errors.insert(num,er)
        num_info_warnings.insert(num,err)
        num_non_doc.insert(num,errr)  
        nm.insert(num,num) 
        num_open_page.insert(num,e)
    end = time ()
    duration = round (end - start, 3)
    minutes = round (duration /60, 1)
    print 'The lists are ready in ', minutes, ' minutes'

In [55]:
#Now we will run the function we created
html_validation (list500_url,list500_names)

The lists are ready in  2.1  minutes


In [56]:
#After the checks we will create the dataframe with the informations we want
d8 = {'company' : pd.Series(list500_names, index=[nm]),'url' : pd.Series(list500_url, index=[nm]),
      'The_page_opened' : pd.Series(num_open_page, index=[nm]),'number_of_errors' : pd.Series(num_errors, index=[nm]),
      'number_of_warning' : pd.Series(num_info_warnings, index=[nm]),'non-document-error' : pd.Series(num_non_doc, index=[nm])}
html_val = pd.DataFrame(d8)    
html_val.head(3) 

,The_page_opened,company,non-document-error,number_of_errors,number_of_warning,url
0,True,Walmart,0,734,1,www.walmart.com
1,True,Exxon Mobil,0,55,29,www.exxonmobil.com
2,True,Apple,0,14,6,www.apple.com


In [57]:
#The next step is to take some informations from the fortune 500 site for each company
#In order to achieve that we should open the pages for each one of the sites seperately
#Since there is a pattern in the way the pages are named it shouldn't be difficult
#Firstly we should create the pattern with which we will download the pages
#By running the code we can see that the names of each comany are not written exactly as we have saved them
#So we do need to alter the names first in order for the below function to run

In [58]:
#creating a new list with alterations in order for the names
#to match the ones that fortune 500 uses so that we can download the html page
list_company_name_new = []
for num in range (0,500):
    cn = list_company_name[num]
    cn = cn.replace(" ", "-")
    cn = cn.replace("&", "")
    cn = cn.replace("’", "")
    cn = cn.replace(".", "-")
    cn = cn.replace("amp;", "")    
    company = cn.lower()
    list_company_name_new.insert(num,cn)

In [59]:
fortune_pages = []
def fortune500 (list_company_name_new):
    from time import time # I used it to see how much time it does to run the function
    start = time ()
    for num3 in range (0,25): #we put 25 for testing
        i = str (num3 +1)    
        companyname =  list_company_name_new[num3]
        browser = urllib2.build_opener() #because i work from different computers with different pyhton version some commands are not recognizable in each version
        browser.addheaders = [('User-agent', 'Mozilla/5.0')]
        site_fortune = "http://beta.fortune.com/fortune500/"+companyname+"-"+ i    
        page_fortune = browser.open(site_fortune)
        html_fortune = page_fortune.read()    
        print("fortune page for company: ", list_company_name_new[num3],i)
        fortune_pages.insert(num3, html_fortune)
    end = time ()
    duration = round (end - start, 3)
    minutes = round (duration /60, 1)
    print 'The lists are ready in ', minutes, ' minutes'
    print 'The lists are ready in ', duration, ' seconds'

In [60]:
#Run the function we created
fortune500 (list_company_name_new)

('fortune page for company: ', 'Walmart', '1')
('fortune page for company: ', 'Exxon-Mobil', '2')
('fortune page for company: ', 'Apple', '3')
('fortune page for company: ', 'Berkshire-Hathaway', '4')
('fortune page for company: ', 'McKesson', '5')
('fortune page for company: ', 'UnitedHealth-Group', '6')
('fortune page for company: ', 'CVS-Health', '7')
('fortune page for company: ', 'General-Motors', '8')
('fortune page for company: ', 'Ford-Motor', '9')
('fortune page for company: ', 'ATT', '10')
('fortune page for company: ', 'General-Electric', '11')
('fortune page for company: ', 'AmerisourceBergen', '12')
('fortune page for company: ', 'Verizon', '13')
('fortune page for company: ', 'Chevron', '14')
('fortune page for company: ', 'Costco', '15')
('fortune page for company: ', 'Fannie-Mae', '16')
('fortune page for company: ', 'Kroger', '17')
('fortune page for company: ', 'Amazon-com', '18')
('fortune page for company: ', 'Walgreens-Boots-Alliance', '19')
('fortune page for comp

In [61]:
#Now that we have opened the url we are going to extract some informations that we need from them
#In order to do that initially we have to create the variables we will need
keyf =[]
per =[]
rev_dol = []
rev_per = []
prof_dol = []
prof_per = []
assets_dol = []
assets_per = []
tse_dol = []
tse_per = []
mar_dol = []
mar_per = []
market = []
nm = []
ln = []
urln = []
empty = []

In [62]:
def fortune_metrics (list_company_name,list_company_website):
    for n in range (0,25):   #we put 25 for testing
        nm.insert(n,n)
        ln.insert(n,list_company_name[n])
        urln.insert(n,list_company_website[n])
        files = fortune_pages[n]
        soup = BeautifulSoup(files,"lxml")
        o=0
        for row in soup.html.body.findAll('tbody'):
            keyf.insert(o,row)
            o=o+1
        keyfin = keyf[0] #the elements we need is in the first tbody of the code
        data = keyfin.findAll('td')

        one = str(data[0]) # revenue
        two = str(data[1]) # revenue in dollars we need to extract this
        revdol= re.findall('>\$(.+?)</td>',two) #we keep only the numbers
        if revdol[0] != empty:
            w = revdol[0]
            a = w.replace("[", "")
            r = a.replace("]","")
            rev_dol.insert(n,r)
        tria = str(data[2])# revenue in percentage we need to extract this as well
        revper= re.findall('>(.+?)%</td>',tria) #we keep only the numbers
        if revper != empty:    
            w = revper[0]
            a = w.replace("[", "")
            r1 = a.replace("]","")    
            rev_per.insert(n,r1) 

        four = str(data[3])   # profit     
        five = str(data[4])   # profit in dollars we need to extract this   
        profdol= re.findall('>\$(.+?)</td>',five) #we keep only the numbers
        if profdol != empty:
            w = profdol[0]
            a = w.replace("[", "")
            p = a.replace("]","")
            prof_dol.insert(n,p)
        six = str(data[5])    # profit in percentage we need to extract this as well   
        profper = re.findall('>(.+?)%</td>',six) #we keep only the numbers
        if profper != empty:
            w = profper[0]
            a = w.replace("[", "")
            p1 = a.replace("]","")    
            prof_per.insert(n,p1)

        seven = str(data[6]) #assets
        eight = str(data[7]) #assets in dollars we need to extract this
        assetsdol= re.findall('>\$(.+?)</td>',eight) #we keep only the numbers
        if assetsdol != empty:
            w = assetsdol[0]
            a = w.replace("[", "")
            ass = a.replace("]","")
            assets_dol.insert(n,ass)

        ten = str(data[9]) #Total Stockholder Equity ($M)    
        eleven = str(data[10]) #Total Stockholder Equity ($M) in dollars we need to extract this
        tsedol= re.findall('>\$(.+?)</td>',eleven) #we keep only the numbers
        if tsedol != empty:
            w = tsedol[0]
            a = w.replace("[", "")
            ts = a.replace("]","")
            tse_dol.insert(n,ts)

        thirteen = str(data[12]) # market value
        fourteen = str(data[13]) # market value in dollars we need to extract this
        mardol= re.findall('>\$(.+?)</td>',fourteen) #we keep only the numbers
        if mardol != empty:
            w = mardol[0]
            a = w.replace("[", "")
            mar = a.replace("]","")
            mar_dol.insert(n,mar)
    print "The function is complete!"

In [63]:
fortune_metrics (list_company_name,list_company_website)

The function is complete!


In [64]:
d9 = {'company' : pd.Series(ln, index=[nm]),'url' : pd.Series(urln, index=[nm]),
      'Revenues $' : pd.Series(rev_dol, index=[nm]),'Revenues %' : pd.Series(rev_per, index=[nm]),
      'Assets $' : pd.Series(assets_dol, index=[nm]),
      'Total Stockholder Equity $' : pd.Series(tse_dol, index=[nm]),
      'Market value $' : pd.Series(mar_dol, index=[nm])}
fort500 = pd.DataFrame(d9)    
fort500

,Assets $,Market value $,Revenues $,Revenues %,Total Stockholder Equity $,company,url
0,"199,581","215,356","482,130",-0.7,"80,546",Walmart,www.walmart.com
1,"336,758","347,129","246,204",-35.6,"170,811",Exxon Mobil,www.exxonmobil.com
2,"290,479","604,304","233,715",27.9,"119,355",Apple,www.apple.com
3,"552,257","350,279","210,821",8.3,"255,550",Berkshire Hathaway,www.berkshirehathaway.com
4,"53,870","35,945","181,241",31.3,"8,001",McKesson,www.mckesson.com
5,"111,383","122,542","157,107",20.4,"33,830",UnitedHealth Group,www.unitedhealthgroup.com
6,"93,657","113,947","153,290",10.0,"37,196",CVS Health,www.cvshealth.com
7,"194,520","48,543","152,356",-2.3,"39,871",General Motors,www.gm.com
8,"224,925","53,758","149,558",3.8,"28,642",Ford Motor,www.ford.com
9,"402,672","240,943","146,801",10.8,"122,671",AT&amp;T,www.att.com


In [70]:
fort500.merge(html_val, left_on='company', right_on='company', how='outer')


,Assets $,Market value $,Revenues $,Revenues %,Total Stockholder Equity $,company,url_x,The_page_opened,non-document-error,number_of_errors,number_of_warning,url_y
0,"199,581","215,356","482,130",-0.7,"80,546",Walmart,www.walmart.com,True,0.0,734.0,1.0,www.walmart.com
1,"336,758","347,129","246,204",-35.6,"170,811",Exxon Mobil,www.exxonmobil.com,True,0.0,55.0,29.0,www.exxonmobil.com
2,"290,479","604,304","233,715",27.9,"119,355",Apple,www.apple.com,True,0.0,14.0,6.0,www.apple.com
3,"552,257","350,279","210,821",8.3,"255,550",Berkshire Hathaway,www.berkshirehathaway.com,True,0.0,4.0,0.0,www.berkshirehathaway.com
4,"53,870","35,945","181,241",31.3,"8,001",McKesson,www.mckesson.com,True,0.0,13.0,15.0,www.mckesson.com
5,"111,383","122,542","157,107",20.4,"33,830",UnitedHealth Group,www.unitedhealthgroup.com,True,0.0,35.0,3.0,www.unitedhealthgroup.com
6,"93,657","113,947","153,290",10.0,"37,196",CVS Health,www.cvshealth.com,True,0.0,114.0,72.0,www.cvshealth.com
7,"194,520","48,543","152,356",-2.3,"39,871",General Motors,www.gm.com,True,0.0,26.0,1.0,www.gm.com
8,"224,925","53,758","149,558",3.8,"28,642",Ford Motor,www.ford.com,True,0.0,136.0,50.0,www.ford.com
9,"402,672","240,943","146,801",10.8,"122,671",AT&amp;T,www.att.com,True,0.0,39.0,5.0,www.att.com


In [74]:
result = pd.merge(fort500, html_val, how='inner', on=['company', 'company'])
result2 = pd.merge(social_media, fre, how='inner', on=['company', 'company'])
result3 = pd.merge(wordss, sizess, how='inner', on=['company', 'company'])
result4 = pd.merge(images_types, loading_time, how='inner', on=['company', 'company'])
result5 = pd.merge(result, sites_links, how='inner', on=['company', 'company'])
result6 = pd.merge(result5, result2, how='inner', on=['company', 'company'])
result7 = pd.merge(result6, result3, how='inner', on=['company', 'company'])
final = pd.merge(result7, result4, how='inner', on=['company', 'company'])
del final['url_x_x']
del final['url_x_y']
del final['url_y_y']
final

,Assets $,Market value $,Revenues $,Revenues %,Total Stockholder Equity $,company,The_page_opened,non-document-error,number_of_errors,number_of_warning,...,.dib,.gif,.jpe,.jpeg,.jpg,.png,.tif,.tiff,total images,loading time
0,"199,581","215,356","482,130",-0.7,"80,546",Walmart,True,0,734,1,...,0,26,103,103,85,43,12,0,372,0.443
1,"336,758","347,129","246,204",-35.6,"170,811",Exxon Mobil,True,0,55,29,...,0,1,0,0,16,2,4,0,23,3.629
2,"290,479","604,304","233,715",27.9,"119,355",Apple,True,0,14,6,...,0,1,0,0,0,2,0,0,3,0.075
3,"552,257","350,279","210,821",8.3,"255,550",Berkshire Hathaway,True,0,4,0,...,0,1,0,0,0,0,0,0,1,0.001
4,"53,870","35,945","181,241",31.3,"8,001",McKesson,True,0,13,15,...,0,0,0,0,6,7,0,0,13,0.832
5,"111,383","122,542","157,107",20.4,"33,830",UnitedHealth Group,True,0,35,3,...,0,0,0,0,0,0,0,0,0,0.915
6,"93,657","113,947","153,290",10.0,"37,196",CVS Health,True,0,114,72,...,0,1,1,0,1,16,1,0,20,0.118
7,"194,520","48,543","152,356",-2.3,"39,871",General Motors,True,0,26,1,...,0,0,0,0,17,92,8,0,117,0.037
8,"224,925","53,758","149,558",3.8,"28,642",Ford Motor,True,0,136,50,...,0,1,0,0,104,6,22,0,133,0.104
9,"402,672","240,943","146,801",10.8,"122,671",AT&amp;T,True,0,39,5,...,1,1,0,0,20,28,5,0,55,0.553
